In [ ]:
import numpy as np
import scipy.ndimage
from skimage import measure, morphology
import SimpleITK as sitk
from multiprocessing import Pool
import os
import nrrd
from scipy.ndimage import label, generate_binary_structure, binary_dilation
from matplotlib import pyplot as plt
import joblib
from scipy.ndimage import binary_fill_holes
import skimage


In [ ]:

def extract_main(binary_mask, cover=0.95):
    """
    Extract lung without bronchi/trachea. Remove small components
    binary_mask: 3D binary numpy array with the same shape of the image,
        that only region of interest is True. One side of the lung in this
        specifical case.
    cover: float, percetange of the total area to keep of each slice, by
        keeping the total connected components
    return: binary mask with the same shape of the image, that only region of
        interest is True. One side of the lung in this specifical case.
    """
 
    for i in range(binary_mask.shape[0]):
        slice_binary = binary_mask[i]
        slice_binary = slice_binary.astype(int)
        label = measure.label(slice_binary)
        properties = measure.regionprops(label)
        properties.sort(key=lambda x: x.area, reverse=True)
        areas = [prop.area for prop in properties]
        count = 0
        area_sum = 0
        area_cover = np.sum(areas) * cover
 
        # count how many components covers, e.g 95%, of total area (lung)
        while area_sum < area_cover:
            area_sum += areas[count]
            count += 1
 
        # SLICE-WISE: exclude trachea/bronchi.
        # only keep pixels in convex hull of big components, since
        # convex hull contains small components of lungs we still want
        slice_filter = np.zeros(slice_binary.shape, dtype=bool)
        for j in range(count):
            min_row, min_col, max_row, max_col = properties[j].bbox
            slice_filter[min_row:max_row, min_col:max_col] |= \
                properties[j].convex_image
 
        binary_mask[i] = binary_mask[i] & slice_filter
    binary_mask = binary_mask.astype(int)
    label = measure.label(binary_mask)
    properties = measure.regionprops(label)
    properties.sort(key=lambda x: x.area, reverse=True)
    # VOLUME: Return lung, ie the largest component.
    binary_mask = (label == properties[0].label)
 
    return binary_mask
 
 
def fill_2d_hole(binary_mask):
    """
    Fill in holes of binary single lung slicewise.
    binary_mask: 3D binary numpy array with the same shape of the image,
        that only region of interest is True. One side of the lung in this
        specifical case.
    return: binary mask with the same shape of the image, that only region of
        interest is True. One side of the lung in this specifical case.
    """
 
    for i in range(binary_mask.shape[0]):
        slice_binary = binary_mask[i]
        slice_binary = slice_binary.astype(int)
        label = measure.label(slice_binary)
        properties = measure.regionprops(label)
 
        for prop in properties:
            min_row, min_col, max_row, max_col = prop.bbox
            slice_binary[min_row:max_row, min_col:max_col] |= \
                prop.filled_image  # 2D component without holes
 
        binary_mask[i] = slice_binary
 
    return binary_mask
 
def seperate_two_lung(binary_mask, spacing, max_iter=22, max_ratio=4.8):
    """
    Gradually erode binary mask until lungs are in two separate components
    (trachea initially connects them into 1 component) erosions are just used
    for distance transform to separate full lungs.
    binary_mask: 3D binary numpy array with the same shape of the image,
        that only region of interest is True.
    spacing: float * 3, raw CT spacing in [z, y, x] order.
    max_iter: max number of iterations for erosion.
    max_ratio: max ratio allowed between the volume differences of two lungs
    return: two 3D binary numpy array with the same shape of the image,
        that only region of interest is True. Each binary mask is ROI of one
        side of the lung.
    """
    found = False
    iter_count = 0
    binary_mask_full = np.copy(binary_mask)
 
    while not found and iter_count < max_iter:
        binary_mask = binary_mask.astype(int)
        label = measure.label(binary_mask, connectivity=2)
        properties = measure.regionprops(label)
        # sort componets based on their area
        properties.sort(key=lambda x: x.area, reverse=True)
        if len(properties) > 1 and \
                properties[0].area / properties[1].area < max_ratio:
            found = True
            # binnary mask for the larger eroded lung
            eroded1 = (label == properties[0].label)
            # binnary mask for the smaller eroded lung
            eroded2 = (label == properties[1].label)
        else:
            # erode the convex hull of each 3D component by 1 voxel
            binary_mask = scipy.ndimage.binary_erosion(binary_mask)
            iter_count += 1
 
    # because eroded lung will has smaller volums than the original lung,
    # we need to label those eroded voxel based on their distances to the
    # two eroded lungs.
    if found:
        # distance1 has the same shape as the 3D CT image, each voxel contains
        # the euclidient distance from the voxel to the closest voxel within
        # eroded1, so voxel within eroded1 will has distance 0.
        distance1 = scipy.ndimage.distance_transform_edt(~eroded1, sampling=spacing)
        distance2 = scipy.ndimage.distance_transform_edt(~eroded2, sampling=spacing)
 
        # Original mask & lung1 mask
        binary_mask1 = binary_mask_full & (distance1 < distance2)
        # Original mask & lung2 mask
        binary_mask2 = binary_mask_full & (distance1 > distance2)
 
        # remove bronchi/trachea and other small components
        binary_mask1 = extract_main(binary_mask1)
        binary_mask2 = extract_main(binary_mask2)
    else:
        # did not seperate the two lungs, use the original lung as one of them
        binary_mask1 = binary_mask_full
        binary_mask2 = np.zeros(binary_mask.shape).astype('bool')
 
    binary_mask1 = fill_2d_hole(binary_mask1)
    binary_mask2 = fill_2d_hole(binary_mask2)
 
    return binary_mask1, binary_mask2
 
def binarize(image, spacing, intensity_thred=-600, sigma=1.0, area_thred=30.0,
             eccen_thred=0.99, corner_side=10):
    """
    Binarize the raw 3D CT image slice by slice
    image: 3D numpy array of raw HU values from CT series in [z, y, x] order.
    spacing: float * 3, raw CT spacing in [z, y, x] order.
    intensity_thred: float, thredhold for lung and air
    sigma: float, standard deviation used for Gaussian filter smoothing.
    area_thred: float, min threshold area measure in mm.
    eccen_thred: float, eccentricity thredhold measure how round is an ellipse
    corner_side: int, side length of top-left corner in each slice,
        in terms of pixels.
    return: binary mask with the same shape of the image, that only region of
        interest is True.
    """
    binary_mask = np.zeros(image.shape, dtype=bool)
    side_len = image.shape[1]  # side length of each slice, e.g. 512
 
    # [-side_len/2, side_len/2], e.g. [-255.5, -254.5, ..., 254.5, 255.5]
    grid_axis = np.linspace(-side_len / 2 + 0.5, side_len / 2 - 0.5, side_len)
 
    x, y = np.meshgrid(grid_axis, grid_axis)
 
    #  pixel distance from each pixel to the origin of the slice of shape
    #  [side_len, side_len]
    distance = np.sqrt(np.square(x) + np.square(y))
 
    # four corners are 0, elsewhere are 1
    nan_mask = (distance < side_len / 2).astype(float)
    nan_mask[nan_mask == 0] = np.nan  # assing 0 to be np.nan
 
    # binarize each slice
    for i in range(image.shape[0]):
        slice_raw = np.array(image[i]).astype('float32')
 
        # number of differnet values in the top-left corner
        num_uniq = len(np.unique(slice_raw[0:corner_side, 0:corner_side]))
 
        # black corners out-of-scan, make corners nan before Gaussian filtering
        # (to make corners False in mask)
        if num_uniq == 1:
            slice_raw *= nan_mask
 
        slice_smoothed = scipy.ndimage.gaussian_filter(slice_raw, sigma,
                                                       truncate=2.0)
 
        # mask of low-intensity pixels (True = lungs, air)
        slice_binary = slice_smoothed < intensity_thred
 
        # get connected componets annoated by label
        slice_binary = slice_binary.astype(int)
        # print(type(slice_binary))
        label = measure.label(slice_binary)
        # print('D')
        properties = measure.regionprops(label)
        label_valid = set()
 
        for prop in properties:
            # area of each componets measured in mm
            area_mm = prop.area * spacing[1] * spacing[2]
 
            # only include comppents with curtain min area and round enough
            if area_mm > area_thred and prop.eccentricity < eccen_thred:
                label_valid.add(prop.label)
 
        # test each pixel in label is in label_valid or not and add those True
        # into slice_binary
        slice_binary = np.in1d(label, list(label_valid)).reshape(label.shape)
        binary_mask[i] = slice_binary
 
    return binary_mask
 
 
def exclude_corner_middle(label):
    """
    Exclude componets that are connected to the 8 corners and the middle of
        the 3D image
    label: 3D numpy array of connected component labels with same shape of the
        raw CT image
    return: label after setting those components to 0
    """
    # middle of the left and right lungs
    mid = int(label.shape[2] / 2)
 
    corner_label = set([label[0, 0, 0],
                        label[0, 0, -1],
                        label[0, -1, 0],
                        label[0, -1, -1],
                        label[-1, 0, 0],
                        label[-1, 0, -1],
                        label[-1, -1, 0],
                        label[-1, -1, -1]])
 
    middle_label = set([label[0, 0, mid],
                        label[0, -1, mid],
                        label[-1, 0, mid],
                        label[-1, -1, mid]])
 
    for l in corner_label:
        label[label == l] = 0
 
    for l in middle_label:
        label[label == l] = 0
 
    return label
 
def volume_filter(label, spacing, vol_min=0.2, vol_max=8.2):
    """
    Remove volumes too large/small to be lungs takes out most of air around
    body.
    adult M total lung capacity is 6 L (3L each)
    adult F residual volume is 1.1 L (0.55 L each)
    label: 3D numpy array of connected component labels with same shape of the
        raw CT image.
    spacing: float * 3, raw CT spacing in [z, y, x] order.
    vol_min: float, min volume of the lung
    vol_max: float, max volume of the lung
    """
    properties = measure.regionprops(label)
 
    for prop in properties:
        if prop.area * spacing.prod() < vol_min * 1e6 or \
           prop.area * spacing.prod() > vol_max * 1e6:
            label[label == prop.label] = 0
 
    return label
 
 
def exclude_air(label, spacing, area_thred=3e3, dist_thred=62):
    """
    Select 3D components that contain slices with sufficient area that,
    on average, are close to the center of the slice. Select component(s) that
    passes this condition:
    1. each slice of the component has significant area (> area_thred),
    2. average min-distance-from-center-pixel < dist_thred
    should select lungs, which are closer to center than out-of-body spaces
    label: 3D numpy array of connected component labels with same shape of the
        raw CT image.
    spacing: float * 3, raw CT spacing in [z, y, x] order.
    area_thred: float, sufficient area
    dist_thred: float, sufficient close
    return: binary mask with the same shape of the image, that only region of
        interest is True. has_lung means if the remaining 3D component is lung
        or not
    """
    # prepare a slice map of distance to center
    y_axis = np.linspace(-label.shape[1]/2+0.5, label.shape[1]/2-0.5,
                         label.shape[1]) * spacing[1]
    x_axis = np.linspace(-label.shape[2]/2+0.5, label.shape[2]/2-0.5,
                         label.shape[2]) * spacing[2]
    y, x = np.meshgrid(y_axis, x_axis)
 
    # real distance from each pixel to the origin of a slice
    distance = np.sqrt(np.square(y) + np.square(x))
    distance_max = np.max(distance)
 
    # properties of each 3D componet.
    vols = measure.regionprops(label)
    label_valid = set()
 
    for vol in vols:
        # 3D binary matrix, only voxels within label matches vol.label is True
        single_vol = (label == vol.label)
 
        # measure area and min_dist for each slice
        # min_distance: distance of closest voxel to center
        # (else max(distance))
        slice_area = np.zeros(label.shape[0])
        min_distance = np.zeros(label.shape[0])
 
        for i in range(label.shape[0]):
            slice_area[i] = np.sum(single_vol[i]) * np.prod(spacing[1:3])
            min_distance[i] = np.min(single_vol[i] * distance +
                                     (1 - single_vol[i]) * distance_max)
 
            # 1. each slice of the component has enough area (> area_thred)
            # 2. average min-distance-from-center-pixel < dist_thred
            if np.average([min_distance[i] for i in range(label.shape[0])
                          if slice_area[i] > area_thred]) < dist_thred:
                label_valid.add(vol.label)
 
    binary_mask = np.in1d(label, list(label_valid)).reshape(label.shape)
    has_lung = len(label_valid) > 0
 
    return binary_mask, has_lung
 
 
def fill_hole(binary_mask):
    """
    Fill in 3D holes. Select every component that isn't touching corners.
    binary_mask: 3D binary numpy array with the same shape of the image,
        that only region of interest is True.
    """
    # 3D components that are not ROI
    binary_mask = binary_mask.astype((int))
    label = measure.label(~binary_mask)
 
    # idendify corner components
    corner_label = set([label[0, 0, 0],
                        label[0, 0, -1],
                        label[0, -1, 0],
                        label[0, -1, -1],
                        label[-1, 0, 0],
                        label[-1, 0, -1],
                        label[-1, -1, 0],
                        label[-1, -1, -1]])
    binary_mask = ~np.in1d(label, list(corner_label)).reshape(label.shape)
 
    return binary_mask
 
 
 
def extract_lung(image, spacing):
    """
    Preprocess pipeline for extracting the lung from the raw 3D CT image.
    image: 3D numpy array of raw HU values from CT series in [z, y, x] order.
    spacing: float * 3, raw CT spacing in [z, y, x] order.
    return: two 3D binary numpy array with the same shape of the image,
        that only region of interest is True. Each binary mask is ROI of one
        side of the lung. Also return if lung is found or not.
    """
    # binary mask with the same shape of the image, that only region of
    # interest is True.
    binary_mask = binarize(image, spacing)
 
    # labelled 3D connected componets, with the same shape as image. each
    # commponet has a different int number > 0
    binary_mask = binary_mask.astype(int)
    label = measure.label(binary_mask, connectivity=1)
 
    # exclude componets that are connected to the 8 corners and the middle
    # of the 3D image
    label = exclude_corner_middle(label)
 
    # exclude componets that are too small or too large to be lung
    # label = volume_filter(label, spacing)
 
    # exclude more air chunks and grab lung mask region
    binary_mask, has_lung = exclude_air(label, spacing)
 
    # fill in 3D holes. Select every component that isn't touching corners.
    binary_mask = fill_hole(binary_mask)
 
    # seperate two lungs
    binary_mask1, binary_mask2 = seperate_two_lung(binary_mask, spacing)
 
    return binary_mask1, binary_mask2, has_lung
 
def load_itk_image(filename):
    """Return img array and [z,y,x]-ordered origin and spacing
    """
 
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
 
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
 
    return numpyImage, numpyOrigin, numpySpacing
 
def resample(image, spacing, new_spacing=[1.0, 1.0, 1.0], order=1):
    """
    Resample image from the original spacing to new_spacing, e.g. 1x1x1
    image: 3D numpy array of raw HU values from CT series in [z, y, x] order.
    spacing: float * 3, raw CT spacing in [z, y, x] order.
    new_spacing: float * 3, new spacing used for resample, typically 1x1x1,
        which means standardizing the raw CT with different spacing all into
        1x1x1 mm.
    order: int, order for resample function scipy.ndimage.interpolation.zoom
    return: 3D binary numpy array with the same shape of the image after,
        resampling. The actual resampling spacing is also returned.
    """
    # shape can only be int, so has to be rounded.
    new_shape = np.round(image.shape * spacing / new_spacing)
 
    # the actual spacing to resample.
    resample_spacing = spacing * image.shape / new_shape
 
    resize_factor = new_shape / image.shape
 
    image_new = scipy.ndimage.zoom(image, resize_factor, mode='nearest', order=order)
 
    return (image_new, resample_spacing)
 
import pydicom
def get_CT_info(src_dir):
    Slices = []
    for file in os.listdir(src_dir):
        if file.endswith('.dcm'):
            slice = pydicom.read_file(src_dir + '/' + file)
            # print ("AAA:", Slices)
            Slices.append(slice)
    Slices.sort(key=lambda x: int(x.InstanceNumber))  # 层序列号
    return Slices
 
 
def get_pixels_hu(Slices):
    images = np.stack([s.pixel_array for s in Slices])
    images_temp = images
    images_temp = images_temp.astype("int32")
    print("Start Dicom pixel_array:", images_temp.dtype)
    for slice_number in range(len(Slices)):
        intercept = Slices[slice_number].RescaleIntercept
        slope = Slices[slice_number].RescaleSlope
        images_temp[slice_number] = slope * images_temp[slice_number] + intercept
    return images_temp
 


In [ ]:
if __name__ == '__main__':
    dcm_dir = '../CT'
    nii_dir = '../CT.nii.gz'
    lstFilesDCM = os.listdir(dcm_dir)
 
    RefDs = pydicom.read_file(os.path.join(dcm_dir, lstFilesDCM[0])) 
    ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM))  
    ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))
    spacing = np.array(ConstPixelSpacing, dtype=float)[::-1]
    Origin = RefDs.ImagePositionPatient
    print(spacing, type(spacing))
    info = sitk.ReadImage(nii_dir)
    img_org = sitk.GetArrayFromImage(info)
    print(img_org.shape)
    binary_mask1, binary_mask2, has_lung = extract_lung(img_org, spacing)
    maskL = np.where(binary_mask1[:] == 1., 170, 0)
    maskR = np.where(binary_mask2[:] == 1., 255, 0)
    

    maskRL = maskL + maskR
    print(binary_mask1.shape)
    print(binary_mask2.shape)
    print(has_lung)
    joblib.dump(maskRL,os.path.join('./output','mask.pkl'))
 


In [ ]:
if __name__ == '__main__':
    path = '../data'
    for i in os.listdir(path):
        if i != "0" and i !="1":
            continue
        folder_a = os.listdir(os.path.join(path,i))
        for folder in folder_a:
            print(folder)
            imagePath = os.path.join(path,i,folder,'CT')
            dcm_dir = imagePath
            nii_dir = os.path.join(path,i,folder,'CT.nii.gz')
            lstFilesDCM = os.listdir(dcm_dir)
            RefDs = pydicom.read_file(os.path.join(dcm_dir, lstFilesDCM[0]))  
            ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM)) 
        
            ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))
            spacing = np.array(ConstPixelSpacing, dtype=float)[::-1]
            Origin = RefDs.ImagePositionPatient
            info = sitk.ReadImage(nii_dir)
            img_org = sitk.GetArrayFromImage(info)
            
            binary_mask1, binary_mask2, has_lung = extract_lung(img_org, spacing)
            maskL = np.where(binary_mask1[:] == 1., 170, 0)
            maskR = np.where(binary_mask2[:] == 1., 255, 0)
            

            maskRL = maskL + maskR
            if has_lung == False:
                print(folder)
            else:
                joblib.dump(maskRL,os.path.join(path,i,folder,'LUNG_mask.pkl'))
        
